## importing important libraries

In [25]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import string
from sklearn.preprocessing import LabelEncoder
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report

In [13]:
files = list(glob("..\second\data\stories_*.csv"))
df = pd.concat([pd.read_csv(file) for file in files] , ignore_index=True)

In [15]:
df.head()

,Unnamed: 0,id,title,date,author,story,topic
0,0,f06aa998054e11eba66e646e69d991ea,"""بيت الشعر"" يسائل وزير الثقافة عن كوابيس سوداء",الجمعة 02 أكتوبر 2020 - 23:19,هسبريس من الرباط,"وجه ""بيت الشعر في المغرب"" إلى وزير الثقافة وال...",art-et-culture
1,1,f1cf1b9c054e11ebb718646e69d991ea,"مهرجان ""سينما المؤلّف"" يستحضر روح ثريا جبران",الجمعة 02 أكتوبر 2020 - 07:26,هسبريس من الرباط,في ظلّ استمرار حالة الطوارئ الصحية المرتبطة بج...,art-et-culture
2,2,f2d282a4054e11eb800f646e69d991ea,"فيلم ""بدون عنف"" لهشام العسري ..""كعب الحذاء ووا...",الجمعة 02 أكتوبر 2020 - 04:00,عفيفة الحسينات*,تشير مشاهدة فيلم قصير ضمن الثلاثية الأخيرة للم...,art-et-culture
3,3,f3f46cac054e11eba403646e69d991ea,"""تنين ووهان"" .. مريم أيت أحمد توقِّع أولى ""روا...",الجمعة 02 أكتوبر 2020 - 02:00,حاورَها: وائل بورشاشن,"مِن قَلب أيّام ""الحَجْر""، رأتِ النّورَ الفصول ...",art-et-culture
4,4,f50f0476054e11eba31b646e69d991ea,"مسكر يتخلّى عن دعم ""الوزارة"" بسبب ""الجمهور""",الخميس 01 أكتوبر 2020 - 19:40,هسبريس من الرباط,أعلن الفنان المغربيّ سعيد مسكر تخليه عن مبلغ ا...,art-et-culture


In [16]:
df.drop(['Unnamed: 0', 'id' , "date" , "title" , "author" ], axis=1 , inplace=True)
df.head()

,story,topic
0,"وجه ""بيت الشعر في المغرب"" إلى وزير الثقافة وال...",art-et-culture
1,في ظلّ استمرار حالة الطوارئ الصحية المرتبطة بج...,art-et-culture
2,تشير مشاهدة فيلم قصير ضمن الثلاثية الأخيرة للم...,art-et-culture
3,"مِن قَلب أيّام ""الحَجْر""، رأتِ النّورَ الفصول ...",art-et-culture
4,أعلن الفنان المغربيّ سعيد مسكر تخليه عن مبلغ ا...,art-et-culture


In [19]:
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation

stop_words = stopwords.words()

arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

def preprocess(text):
    
    '''
    text is an arabic string input
    
    the preprocessed text is returned
    '''
    
    #remove punctuations
    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)
    
    # remove Tashkeel
    text = re.sub(arabic_diacritics, '', text)
    
    #remove longation
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)

    text = ' '.join(word for word in text.split() if word not in stop_words)

    return text
  
df['story'] = df['story'].apply(preprocess)

In [23]:
labeling = LabelEncoder()
labeling.fit(df['topic'])
df['target'] = labeling.transform(df['topic'])


,story,topic,target
0,وجه بيت الشعر المغرب الي وزير الثقافه والشباب ...,art-et-culture,0
1,ظل استمرار حاله الطوارء الصحيه المرتبطه بجاءحه...,art-et-culture,0
2,تشير مشاهده فيلم قصير ضمن الثلاثيه الاخيره للم...,art-et-culture,0
3,قلب ايام الحجر رات النور الفصول الاولي روايه م...,art-et-culture,0
4,اعلن الفنان المغربي سعيد مسكر تخليه مبلغ الدعم...,art-et-culture,0
...,...,...,...
10995,دافع ناشطون امازيغ استقبلوا اخيرا وفدا الطلبه ...,tamazight,10
10996,خاضت الشاعره الامازيغيه ملكيه مزان اعتصاما لمد...,tamazight,10
10997,ادانت المنسقيه الوطنيه للمبادره الطلابيه ضد ال...,tamazight,10
10998,طالبت الناشطه الامازيغيه مريم الدمناتي بضروره ...,tamazight,10


In [24]:
df.drop(['topic' ], axis=1 , inplace=True)
df.head()

,story,target
0,وجه بيت الشعر المغرب الي وزير الثقافه والشباب ...,0
1,ظل استمرار حاله الطوارء الصحيه المرتبطه بجاءحه...,0
2,تشير مشاهده فيلم قصير ضمن الثلاثيه الاخيره للم...,0
3,قلب ايام الحجر رات النور الفصول الاولي روايه م...,0
4,اعلن الفنان المغربي سعيد مسكر تخليه مبلغ الدعم...,0


In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(df.story, df.target, test_size =.2, random_state=42)

In [33]:

model = make_pipeline(TfidfVectorizer(),
                    LogisticRegression())

model.fit(X_train,Y_train)

prediction = model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction))

Accuracy score is 0.85
              precision    recall  f1-score   support

           0       0.83      0.90      0.86       206
           1       0.81      0.84      0.82       202
           2       0.94      0.96      0.95       184
           3       0.86      0.84      0.85       214
           4       0.97      0.89      0.93       197
           5       0.69      0.74      0.71       204
           6       0.78      0.79      0.78       210
           7       0.74      0.86      0.80       178
           8       0.77      0.68      0.72       198
           9       0.99      0.97      0.98       194
          10       0.98      0.85      0.91       213

    accuracy                           0.85      2200
   macro avg       0.85      0.85      0.85      2200
weighted avg       0.85      0.85      0.85      2200



In [34]:
model2 = make_pipeline(TfidfVectorizer(),
                    RandomForestClassifier())

model2.fit(X_train,Y_train)

prediction = model2.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")

Accuracy score is 0.81


In [35]:
pipe = make_pipeline(TfidfVectorizer(),
                    MultinomialNB())
pipe.fit(X_train,Y_train)
prediction = pipe.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction)) 

Accuracy score is 0.80
              precision    recall  f1-score   support

           0       0.89      0.77      0.82       206
           1       0.80      0.87      0.84       202
           2       0.96      0.90      0.93       184
           3       0.89      0.70      0.78       214
           4       0.55      0.95      0.70       197
           5       0.70      0.66      0.68       204
           6       0.79      0.80      0.80       210
           7       0.85      0.76      0.80       178
           8       0.72      0.59      0.65       198
           9       0.99      0.97      0.98       194
          10       0.92      0.89      0.90       213

    accuracy                           0.80      2200
   macro avg       0.82      0.81      0.81      2200
weighted avg       0.82      0.80      0.81      2200



In [36]:
pipe = make_pipeline(TfidfVectorizer(),
                     SVC())

pipe.fit(X_train, Y_train)

prediction = pipe.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction))

Accuracy score is 0.85
              precision    recall  f1-score   support

           0       0.84      0.90      0.87       206
           1       0.83      0.84      0.83       202
           2       0.95      0.95      0.95       184
           3       0.87      0.86      0.87       214
           4       0.97      0.89      0.93       197
           5       0.66      0.76      0.71       204
           6       0.81      0.80      0.80       210
           7       0.80      0.85      0.82       178
           8       0.76      0.72      0.74       198
           9       0.99      0.97      0.98       194
          10       0.98      0.86      0.91       213

    accuracy                           0.85      2200
   macro avg       0.86      0.86      0.86      2200
weighted avg       0.86      0.85      0.86      2200

